# Trump Tweet URLs

So Trump's Twitter account is gone. At least at twitter.com. But (fortunately for history) there has probably never been a more heavily archived social media account at the Internet Archive and elsewhere on the web. There are also a plethora of online "archives" like [The Trump Archive](https://www.thetrumparchive.com/). You can go view the account and [browse around](https://web.archive.org/web/20210107055108/https://twitter.com/realDonaldTrump) but what if we wanted a list of all the Trump tweets?

## CDX API

The Wayback Machine (and many other web archives) have a service called the [CDX API](https://github.com/internetarchive/wayback/tree/master/wayback-cdx-server). Think of it as the index to the archive. You can give it a URL and it'll tell you what snapshots it has for it. You can also ask the CDX API to search for a *url prefix* and it will tell you what snapshots it has that start with that string. Lets use the handy [wayback](https://wayback.readthedocs.io/en/stable/usage.html) Python module to search for tweet URLs in the Wayback machine. So URLs that look like:

    https://twitter.com/realDonaldTrump/status/{id}

In [121]:
! pip install wayback

The search() method handles paging through the API results using a resumption token behind the scenes. Lets look at the first 100 results just to see what they look like.

In [24]:
from wayback import WaybackClient

wb = WaybackClient()

count = 0
for result in wb.search('twitter.com/realDonaldTrump/status/', matchType='prefix'):
    print(result.url)
    count += 1
    if count > 100:
        break

https://twitter.com/realDonaldTrump/status/%22/6ixpercent/status/642559156784775168%22
https://twitter.com/realDonaldTrump/status/%22/6ixpercent/status/645771399726919680%22
https://twitter.com/realdonaldtrump/status/%22/BeauJohnson4/status/481467498248671233%22
https://twitter.com/realDonaldTrump/status/%22/bobby_stokes/status/497930146968395776%22
https://twitter.com/realDonaldTrump/status/%22/CharlesWoodso10/status/652578215400046592%22
https://twitter.com/realDonaldTrump/status/%22/El_Grinchh/status/637030969338294272%22
https://twitter.com/realdonaldtrump/status/%22/jefftiedrich/status/481830345814188034%22
https://twitter.com/realDonaldTrump/status/%22/JesusLezz/status/651908920714371072%22
https://twitter.com/realDonaldTrump/status/%22/jimmygarner/status/652537219694108673%22
https://twitter.com/realDonaldTrump/status/%22/LittelCaesar/status/497929766230831104%22
https://twitter.com/realDonaldTrump/status/%22/PattyDs50/status/637067362353704960%22
https://twitter.com/realDonaldT

So there are some weird URLs in there, that look like the result of buggy automated archive processes that aren't constructing URLs properly?

* https://twitter.com/realDonaldTrump/status/%22/SyakibPutera/status/636601131339087872%22
* https://twitter.com/realDonaldTrump/status/'+twitter_id+

And then we can see lots of results for the same URL such as https://twitter.com/realDonaldTrump/status/1000061992042975232 repeated over and over. This is because that URL was archived at multiple times. So lets improve on this to filter out the URLs that don't look like tweet URLs, and to only emit the unique ones. But still we'll just look at the first 100 results.

In [25]:
import re

seen = set()

for result in wb.search('twitter.com/realDonaldTrump/status/', matchType='prefix'):
    if re.search(r'/realDonaldTrump/status/\d+', result.url):
        if result.url not in seen:
            print(result.url)
            seen.add(result.url)
            if len(seen) > 100:
                break

https://twitter.com/realDonaldTrump/status/0?s=19
https://twitter.com/realDonaldTrump/status/0n/7
http://twitter.com/realDonaldTrump/status/1000061992042975232
https://twitter.com/realDonaldTrump/status/1000061992042975232
https://www.twitter.com/realDonaldTrump/status/1000061992042975232
https://twitter.com/realDonaldTrump/status/1000061992042975232?lang=en
https://twitter.com/realDonaldTrump/status/1000061992042975232?lang=en-gb
https://twitter.com/realDonaldTrump/status/1000061992042975232?ref_src=twcamp%5Ecopy%7Ctwsrc%5Eandroid%7Ctwgr%5Ecopy%7Ctwcon%5E7090%7Ctwterm%5E3
https://twitter.com/realDonaldTrump/status/1000061992042975232?ref_src=twsrc%5Etfw
http://twitter.com/realDonaldTrump/status/1000064605903876096
https://twitter.com/realDonaldTrump/status/1000064605903876096
https://www.twitter.com/realDonaldTrump/status/1000064605903876096
https://twitter.com/realDonaldTrump/status/1000064605903876096?ref_src=twsrc%5Etfw
http://twitter.com/realDonaldTrump/status/1000114139136606209


This list shows that some tweet URLs can have query strings, which modify the presentation of the tweet in various ways. For example to change the language of the user interface:

* https://twitter.com/realDonaldTrump/status/1000114139136606209?lang=en-gb

Or to highlight certain information:

* https://twitter.com/realDonaldTrump/status/1000114139136606209?conversation_id=1000114139136606209

The query parameters are needed for finding the right view in the Wayback Machine. But the different variants don't really matter if we are simply wanting to count the number of tweets that are archived. Also it looks like some URLs aren't for the tweets themselves, but for components of the tweet, like video:

* https://twitter.com/realDonaldTrump/status/1000114139136606209/video/1

The process can be adjusted to parse the URL to ensure the path is for an actual tweet,not a tweet component. The tweet id can also be extracted from the path in order to track whether it has been seen before.

In [26]:
from urllib.parse import urlparse

seen = set()

for result in wb.search('twitter.com/realDonaldTrump/status/', matchType='prefix'):
    uri = urlparse(result.url)
    
    m = re.match(r'^/realDonaldTrump/status/(\d+)/?$', uri.path, re.IGNORECASE)
    if not m:
        continue
        
    tweet_id = m.group(1)
    if tweet_id not in seen:
        print(result.url)
        seen.add(tweet_id)
            
    if len(seen) > 100:
        break

https://twitter.com/realDonaldTrump/status/0?s=19
http://twitter.com/realDonaldTrump/status/1000061992042975232
http://twitter.com/realDonaldTrump/status/1000064605903876096
http://twitter.com/realDonaldTrump/status/1000114139136606209
http://twitter.com/realDonaldTrump/status/1000138164923781121
http://twitter.com/realDonaldTrump/status/1000145873274359809
http://twitter.com/realDonaldTrump/status/1000151354701213696
http://twitter.com/realDonaldTrump/status/1000152863035805696
http://twitter.com/realDonaldTrump/status/1000174070061813761
http://twitter.com/realDonaldTrump/status/1000366478846300165
http://twitter.com/realDonaldTrump/status/1000375761604370434
http://twitter.com/realDonaldTrump/status/1000385794115325952
http://twitter.com/realDonaldTrump/status/1000387893427351553
http://twitter.com/realDonaldTrump/status/1000391997969092608
http://twitter.com/realDonaldTrump/status/1000396430371106817
http://twitter.com/realDonaldTrump/status/1000458567147839488
http://twitter.com/r

Apart from a weird anomaly https://twitter.com/realDonaldTrump/status/0?s=19 it looks like this is actually working pretty good. For completeness we can store a mapping of the tweet id to all the results for that tweet id. This will allow us to track how many tweet have been archiving, while letting us examing how many times that tweet was archived, and what their precise URLs are for playback.

This time we can let it keep running to get all the results.

In [27]:

from collections import defaultdict

tweets = defaultdict(list)

for result in wb.search('twitter.com/realDonaldTrump/status/', matchType='prefix'):
    uri = urlparse(result.url)
    
    m = re.match(r'^/realDonaldTrump/status/(\d{8,})/?$', uri.path, re.IGNORECASE)
    if not m:
        continue
        
    tweet_id = m.group(1)
    tweets[tweet_id].append(result)

Now we can see the tweet ids. Instead of printing them all out we can just look at the first 100:

In [5]:
list(tweets.keys())[0:100]

['1000061992042975232',
 '1000064605903876096',
 '1000114139136606209',
 '1000138164923781121',
 '1000145873274359809',
 '1000151354701213696',
 '1000152863035805696',
 '1000174070061813761',
 '1000366478846300165',
 '1000375761604370434',
 '1000385794115325952',
 '1000387893427351553',
 '1000391997969092608',
 '1000396430371106817',
 '1000458567147839488',
 '1000462031500099584',
 '1000465814192099330',
 '1000554657859670016',
 '1000718611688943616',
 '1000726832843501570',
 '1000741764565753856',
 '1000831304836018176',
 '1000837182297464832',
 '100089437434477630',
 '1001055932376387584',
 '1001085207825534976',
 '1001099771602456576',
 '1001103028626345984',
 '1001108964216537089',
 '1001149367271686145',
 '1001149793496977408',
 '1001151001699803138',
 '1001212077699149825',
 '1001220050995511298',
 '1001404640796336128',
 '1001410457092218880',
 '1001415199516254208',
 '1001417880116891650',
 '1001420270094168064',
 '1001424695126880258',
 '1001455721588969472',
 '100180717424971

And we can look at when a given tweet was archived too, irrespective of the various query strings that can be part of it. Here we get all the snapshots for tweet id 1002298565299965953 and print out the times that it was archived, in descending order. 

In [6]:
for result in sorted(tweets['1002298565299965953'], key=lambda r: r.timestamp, reverse=True):
    print(result.timestamp)

2021-01-08 13:24:02+00:00
2020-11-25 02:50:00+00:00
2020-11-15 17:00:34+00:00
2020-11-09 02:37:14+00:00
2020-11-09 02:37:14+00:00
2020-11-06 08:20:49+00:00
2020-11-06 05:32:52+00:00
2020-10-11 09:07:44+00:00
2020-07-10 17:13:56+00:00
2020-06-30 18:27:42+00:00
2020-06-30 18:26:24+00:00
2020-06-16 20:52:42+00:00
2020-06-07 07:43:55+00:00
2019-11-10 07:27:23+00:00
2019-09-11 04:17:42+00:00
2019-09-11 04:17:25+00:00
2019-08-25 17:20:01+00:00
2019-08-25 10:14:36+00:00
2019-06-13 09:14:18+00:00
2019-05-13 13:45:24+00:00
2019-05-05 01:44:53+00:00
2018-12-29 05:30:13+00:00
2018-11-27 19:12:03+00:00
2018-08-22 14:21:16+00:00
2018-06-05 13:07:07+00:00
2018-06-05 08:38:29+00:00
2018-06-05 08:38:27+00:00
2018-06-05 08:38:24+00:00
2018-06-05 08:38:22+00:00
2018-06-05 08:38:20+00:00
2018-06-05 08:38:18+00:00
2018-06-05 08:38:16+00:00
2018-06-05 08:38:14+00:00
2018-06-05 08:38:12+00:00
2018-06-05 08:38:10+00:00
2018-06-05 08:38:09+00:00
2018-06-05 08:38:07+00:00
2018-06-05 08:38:05+00:00
2018-06-05 0

In [7]:
len(tweets['1002298565299965953'])

253

So this particular URL was archived 252 times! The snapshots start on May 31, 2018 and most of the snapshots are from a few days of that. But there are also a handful of snapshots in 2019 and 2020. Examining [one of the snapshots] shows that it was sent on May 31st at 2:19 PM. It's hard to tell what time zone the display was generated for. But since the first snapshot was at May 31, 2018 at 21:19:36 UTC it is safe to assume that the display is for -07:00 UTC, or (given the time of year) Pacific Daylight Time.

The [overview](https://web.archive.org/web/20180101000000*/twitter.com/realDonaldTrump/status/1002298565299965953) gives a picture of some of these snapshots. But the nice thing about our index is that it factors in the way that the tweet ID is expressed in the URL. So we know more than what the URL specific overview shows. For example here are all the various URLs that were collected.

In [8]:
for result in sorted(tweets['1002298565299965953'], key=lambda r: r.timestamp, reverse=True):
    print(result.url)

https://twitter.com/realDonaldTrump/status/1002298565299965953
https://twitter.com/realdonaldtrump/status/1002298565299965953
https://twitter.com/realDonaldTrump/status/1002298565299965953
https://twitter.com/realDonaldTrump/status/1002298565299965953
https://www.twitter.com/realDonaldTrump/status/1002298565299965953
https://twitter.com/realDonaldTrump/status/1002298565299965953
https://twitter.com/realdonaldtrump/status/1002298565299965953
https://twitter.com/realDonaldTrump/status/1002298565299965953?ref_src=twsrc%5Etfw
https://twitter.com/realDonaldTrump/status/1002298565299965953
https://twitter.com/realDonaldTrump/status/1002298565299965953
https://twitter.com/realdonaldtrump/status/1002298565299965953
https://twitter.com/realdonaldtrump/status/1002298565299965953
https://twitter.com/realdonaldtrump/status/1002298565299965953
https://twitter.com/realdonaldtrump/status/1002298565299965953
https://twitter.com/realDonaldTrump/status/1002298565299965953
https://twitter.com/realDonaldT

What was the most archived tweet?

In [9]:
sorted(tweets, key=lambda r: len(tweets[r]), reverse=True)[0]

'1006837823469735936'

In [10]:
len(tweets['1006837823469735936'])

23420

So https://twitter.com/realDonaldTrump/status/1006837823469735936 was archived 23,419 times?! It's interesting that the [overview page](https://web.archive.org/web/*/twitter.com/realDonaldTrump/status/1006837823469735936) only says 595 times, because it is looking at that exact URL. Looking at [the content](https://web.archive.org/web/20180613095659/twitter.com/realDonaldTrump/status/1006837823469735936) of the tweet it is understandable why this one was archived so much.

## Missing Data?

So what does the coverage look like? So before Trump's account was suspended [his profile](https://web.archive.org/web/20210107045727/https://twitter.com/realDonaldTrump/) indicated he has sent 59.6K tweets. The [TrumpTweetArchive](https://www.thetrumparchive.com/) also shows 56,571 tweets. How many tweet IDs did we find?

In [11]:
len(tweets)

24289

That is *a lot* less than what we should have found. So either there is a problem with my code, or the wayback module isn't paging results properly, or the CDX API isn't functioning properly, or not all of Trumps tweets have been archived?

In conversation with [Rob Brackett](https://robbrackett.com/) who is the principal author of the Python [wayback](https://pypi.org/project/wayback) library it seems that using the `limit` parameter can help return more results. So instead of doing:

    wb.search('twitter.com/realDonaldTrump/status/', matchType='prefix')

the `limit` parameter should be used:

    wb.search('twitter.com/realDonaldTrump/status/', matchType='prefix', limit=500000)

Rob said:

> Basically what’s happening here is that, without the `limit` parameter, the first page of results hits the maximum size and then, in that situation, does not include a resume key for moving on to the next page. Including a low enough limit (I think anything less than 1.5 million, but not sure) prevents you from hitting that ceiling and lets you successfully page through everything. When I do that, I get 64,329 tweet IDs across 16,253,658 CDX records (but the default Colab instance doesn’t have enough memory to store every record like you’re doing, so I had to just store the first record for each ID).

So lets give this a try. But like Mike says, we're likely to consume all working memory storing all these CDX records in RAM. So lets persist them to a little sqlite database.

In [ ]:
import pathlib
import sqlite3

data = pathlib.Path('data')
db_path = data / "trump-tweets.sqlite3"

if db_path.is_file():
    db_path.unlink()
    
db = sqlite3.connect(db_path)
db.execute(
    '''
    CREATE TABLE tweets (
        tweet_id TEXT,
        url TEXT,
        timestamp DATETIME,
        mime_type TEXT,
        status_code INTEGER,
        digest TEXT,
        length INTEGER
    )
    '''
)


for result in wb.search('twitter.com/realDonaldTrump/status/', matchType='prefix', limit=500000):
    uri = urlparse(result.url)
    
    m = re.match(r'^/realDonaldTrump/status/(\d{8,})/?$', uri.path, re.IGNORECASE)
    if not m:
        continue
        
    tweet_id = m.group(1)    
    db.execute('INSERT INTO tweets VALUES (?, ?, ?, ?, ?, ?, ?)', [
        tweet_id,
        result.url,
        result.timestamp,
        result.mime_type,
        result.status_code,
        result.digest,
        result.length
    ])
    
db.close()